In [1]:
#2024-04-16딥러닝프레임워크

In [ ]:
'''
현재의 DeZero는 1차미분만 자동으로 계산한다. 이것을 N차 미분까지 자동으로 계산하게 하려면
1.현재의 역전파 구현을 근본적으로 재검토해야한다.
2. 현재의 Variable과 Function 클래스 구현에 대한 고찰이 필요하다.
위의 두가지를 고려해야한다. 

<Variable 클래스의 __init__ 메서드>
Variable 클래스에는 인스턴스 변수가 여러개이다. 
그 중 data와 grad 인스타스 변수는 각각 순전파 계산과 역전파 계산 시 사용된다.
주의해야할 점은 data와 grad 모두 ndarray 인스턴스를 저장한다는 것이다. 이 점을 부각하기 위해
data 와 grad는 입방체 상자로 그린다.[그림30-1] 
data 와 grad가 ndarray 인스턴스를 참조하는 경우를 [그림30-2]로 나타냄(data와 grad가 ndarray 인스턴스를 참조)
왼쪽만 채워졌을때는 x = Variable(np.array(2.0))을 실행, 
오른쪽까지 채워진 경우는 x.backward()와 x.grad = np.array(1.0)를 실행한 경우이다.

<Function 클래스의 __call__ 메서드>
코드를 살펴보자.
1. 순전파 계산(메인 처리)
xs = [x.data for x in inputs]에 의해 Variable 인스턴스 변수인 data를 꺼내 리스트 xs로 모은다. 
그리고 forward(*xs)를 호출하여 구체적인 계산을 완료한다.

2.Variable 과 Function 의 관계가 만들어지며 변수에서 함수로의 연결은 set_creator 메서드가 만들어준다. 
새로 생성된 Variable에 부모함수를 알려주는 것이다. 
또한 함수의 입력과 출력 변수를 inputs와 outputs라는 인스턴스 변수에 저장하여 함수에서 변수로의 연결을 유지.
변수와 함수의 연결을 만드는 이유는 미분값을 역방향으로 흘려보내기 때문이다. 
 ex) Sin 클래스 코드
순전파의 구체적인 계산은 sin 클래스의 forward 메서드에서 진행되고
(인수와 반환값은 모두 ndarray 인스턴스이며 backward도 마찬가지)
순전파를 하고나면 변수와 함수의 연결이 만들어지는데, 연결은 __call__ 메서드에서 만들어진다.

'''

In [ ]:
'''
역전파 로직

역전파는 Variable 클래스의 backward 메서드에서 구현한다. 
1. Variable의 인스턴스 변수인 grad를 리스트로 모은다. 
2.backward 메서드에는 ndarray 인스턴스가 담긴 리스트가 전달됨.
3. 출력 쪽에서 전파하는 미분값(gxs)을 함수의 입력 변수(f.inputs)의 grad로 설정함
이 사실을 바탕으로 sin함수를 계산하고 바로 역전파하면 모든 변수가 미분 결과를 메모리에 유지하며
순전파 y = sin(x)가 실행될 때 계산 그래프가 만들어지고, Variable 인스턴스 변수인 data가 채워짐
역전파 시 Sin 클래스의 backward 메서드가 불리고 Variable의 인스턴스 변수인 grad가 채워진다.

'''

In [ ]:
'''
[현재의 DeZero 구현]
계산의 연결은 Function 클래스의 __call__ 메서드에서 만들어진다.
순전파와 역전파 계산은 Function 클래스를 상속한 클래스의 forward 와 backward 메서드로 처리함
[문제점]
계산 그래프의 연결이 만들어지는 시점으로 순전파를 계산할 때 만들어진다.
역전파를 계산할 때는 연결이 만들어지지 않는다.
'''

In [ ]:
'''
<고차미분(이론)>

[역전파 계산]
만약 역전파를 계산할 때도 연결이 만들어진다면 고차 미분을 자동으로 계산할 수 있게 된다.
목표는 미분 계산을 계산그래프로 만드는 것이다. (역전파때 수행되는 계산) 
따라서 역전파 때 수행되는 계산에 대해서도 연결을 만들면 문제가 해결된다.

[역전파로 계산 그래프 만들기]

Variable 인스턴스를 사용하여 순전파를 하는 시점에서 연결이 만들어진다. 
backward( ) 메서드에서 ndarray 인스턴스가 아닌 Variable 인스턴스를 사용하면 계산의 연결이 만들어진다는 뜻이다.

이를 하기위해서는 
1. 미분값(기울기)를 Variable 인스턴스 형태로 유지해야 하며
2. Variable 클래스의 grad는 ndarray 인스턴스를 참조하는 대신 Variable 인스턴스를 참조하도록 변경해야한다.
그러면 역전파 계산에 대한 계산 그래프도 만들어진다는 것이다.

'''

In [ ]:
'''
<고차미분(구현)>
역전파 시 수행되는 계산에 대해서 계산 그래프를 만들고 역전파 시에도 Variable 인스턴스를 사용

[패키지 구조 변경]
1. 지금까지는 Variable 클래스를 dezero/core_simple.py에 구현함
2. 고차 미분을 할수 있는 새로운 Variable 클래스를 dezero/core.py 에 구현
3. dezero/core_simple.py에 구현했던 함수와 연산자 오버로드들 또한 dezero/core.py 에서 새로 구현.

'''

In [2]:
#새로운 DeZero의 가장 중요한 변화. 미분값을 자동으로 저장하는 코드-> self.grad가 Variable인스턴스를 담는 코드

def backward(self, retain_grad=False):
        if self.grad is None:
            #self.grad = np.ones_like(self.data)
            self.grad = Variable(np.ones_like(self.data))

In [ ]:
#mul class의 backward 코드 변경
'''
수정 전에는 variable인스턴스 안에 있는 데이터를 꺼내야했지만
수정 후에는 mul클래스에서 variable 인스턴스를 그대로 사용한다.
'''

class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

def backward(self, gy):
        x0, x1 = self.inputs #수정된 부분
        return gy * x1, gy * x0

In [ ]:
#역전파의 활성/비활성 모드 도입 variable의 backward부분 변경하여 core.py에 저장

def backward(self, retain_grad=False, create_graph=False): 
#create_graph를 추가하여 기본값을 False로 설정. 2차이상의 미분이 필요하다면 true. 실무에서는 역전파 1회실행이 잦음
    ***
     while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs] 

            with using_config('enable_backprop', create_graph): #이곳에서 역전파 처리됨.
                gxs = f.backward(*gys) #메인 backward
                if not isinstance(gxs, tuple):
                    gxs = (gxs,)

                for x, gx in zip(f.inputs, gxs):
                    if x.grad is None:
                        x.grad = gx
                    else:
                        x.grad = x.grad + gx

                    if x.creator is not None:
                        add_func(x.creator)


In [ ]:
'''
 [초기화 수행]
1. __init__.py에서 is_simple_core 플래그 설정
2. is_simple_core = False 로 설정하여 dezero/core.py에서 임포트하도록 변경
3. 고차 미분에 대응하는 core 파일을 임포트할 수 있음
    이제 새로운 DeZero는 2차미분도 자동으로 이차미분을 계산한다. 
'''

In [49]:
if '__file__' in globals():
	import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
출처: https://amber-chaeeunk.tistory.com/56 [채채씨의 학습 기록:티스토리]

import numpy as np
from dezero import Variable
from dezero.core_simple import Variable

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [50]:
#2차 미분 계산하기

def f(x):
    y = x ** 4 - 2 * x ** 2
    return y

x = Variable(np.array(2.0))
y = f(x)
y.backward(create_graph=True)
print(x.grad)
#두번째 역전파
gx = x.grad
gx.backward()
print(x.grad) 

#실행결과 variable(24.0) variable(68.0)
'''
하지만 이 코드에는 문제점이 있다. 미분값이 남아있는 상태에서 새로운 역전파를 수행했기 때문에
새로운 미분값이 더해진 값이 출력된다는 것이다.( 24(1차미분값) + 44(2차미분값) 인 상태)
'''

TypeError: Variable.backward() got an unexpected keyword argument 'create_graph'

In [51]:
#2차 미분 계산하기(2) _ 문제점 해결

def f(x):
    y = x ** 4 - 2 * x ** 2
    return y

x = Variable(np.array(2.0))
y = f(x)
y.backward(create_graph=True)
print(x.grad)

gx = x.grad
x.cleargrad() #미분값 재설정
gx.backward()
print(x.grad) 

#결괏값 variable(24.0) variable(44.0)

TypeError: Variable.backward() got an unexpected keyword argument 'create_graph'

In [52]:
#2차미분 (뉴턴방법 이용하여 최적화)

def f(x):
    y = x ** 4 - 2 * x ** 2
    return y

x = Variable(np.array(2.0))
iters = 10

for i in range(iters):
    print(i, x)

    y = f(x)
    x.cleargrad()
    y.backward(create_graph=True)

    gx = x.grad
    x.cleargrad()
    gx.backward()
    gx2 = x.grad

    x.data -= gx.data / gx2.data

'''
출력값 
variable(2.0) 
variable(1.4545454545454546)
'
'
'
variable(1.0)
'''

0 variable(2.0)


TypeError: Variable.backward() got an unexpected keyword argument 'create_graph'

In [53]:
'''
[새로운 DeZero 함수 추가하기]
1.고차 미분에 대응하는 함수들을 core.py에 구현함
    (Add, Mul, Neg, Sub, Div, Pow 클래스)
2. 새로운 함수는 dezero/functions.py에 추가함
3. 새로운 함수를 다른 파일에서 사용시 from dezero.functions import sin 형태로 임포트 해야함

[고차 미분에 대응하는 새로운 sin함수 구현]
수식 y = sin(x) 일때 y’ = cos(x)
▪ backward 메서드 안의 모든 변수가 Variable 인스턴스임
▪ gx = gy * cos(x) 에서 cos(x)는 DeZero의 cos 함수임
• Sin 클래스를 구현하려면 Cos 클래스와 cos 함수가 필요함
▪ gy * cos(x) 에는 곱셈 연산자를 오버로드해 놓았기 때문에 mul 함수가 호출됨

[고차 미분에 대응하는 새로운 Cos 클래스/ cos 함수 구현]
▪ 수식 y = cos(x) 일때 y’ = -sin(x)
▪ backward 메서드에서 구체적인 계산에서 sin 함수를 사용

[sin 함수의 고차 미분]
▪ 2차 미분뿐 아니라 3차 미분, 4차 미분도 계산
▪ for 문을 사용하여 역전파를 반복하여, n차 미분을 구함
▪ 먼저 gx = x.grad에서 미분값을 꺼내 gx에서 역전파하는 것임
▪ 역전파를 하기 전에 x.cleargrad( )를 호출하여 미분값을 재설정함
▪ 이 작업을 반복하여 n차 미분을 계산

[sin 함수의 고차 미분 그래프 그리기]
▪ x = Variable(np.linspace(-7, 7, 200)) 형태로 설정.
• -7 부터 7까지 균일하게 200 등분한 배열을 만들어 줌.
• 만들어진 1차원 배열을 Variable로 감싸 x에 할당.
▪ 다차원 배열을 입력받으면 각 원소에 대해 독립적으로 계산함.
▪ 한 번의 순전파로 원소 200개의 계산이 모두 이루어짐.
▪ 각각의 그래프는 sin(x) -> cos(x) -> -sin(x) -> -cos(x) 식으로 진행이 되어 위상이 어긋남.

[tanh 함수 추가]
▪ tanh 는 쌍곡탄젠트 혹은 하이퍼볼릭 탄젠트. 입력을 -1 ~ 1 사이 값으로 반환
미분은 y = tanh(x) 일때, y’ = 1 - 𝒚^2 임.
▪ 순전파에서 np.tanh 메서드를 이용함
▪ 역전파에서는 gy * (1 – y * y) 형태로 구현
▪ 재 사용할 수 있도록 dezero/functions.py에 추가

[tanh 함수의 고차 미분 계산 그래프 시각화]
▪ for 문에서 반복해서 역전파함으로 고차 미분을 계산
▪ iters = 0 이면 1차 미분, 1이면 2차 미분이 계산되는 방식임

'''

'\n[새로운 DeZero 함수 추가하기]\n1.고차 미분에 대응하는 함수들을 core.py에 구현함\n    (Add, Mul, Neg, Sub, Div, Pow 클래스)\n2. 새로운 함수는 dezero/functions.py에 추가함\n3. 새로운 함수를 다른 파일에서 사용시 from dezero.functions import sin 형태로 임포트 해야함\n\n[고차 미분에 대응하는 새로운 sin함수 구현]\n수식 y = sin(x) 일때 y’ = cos(x)\n▪ backward 메서드 안의 모든 변수가 Variable 인스턴스임\n▪ gx = gy * cos(x) 에서 cos(x)는 DeZero의 cos 함수임\n• Sin 클래스를 구현하려면 Cos 클래스와 cos 함수가 필요함\n▪ gy * cos(x) 에는 곱셈 연산자를 오버로드해 놓았기 때문에 mul 함수가 호출됨\n\n[고차 미분에 대응하는 새로운 Cos 클래스/ cos 함수 구현]\n▪ 수식 y = cos(x) 일때 y’ = -sin(x)\n▪ backward 메서드에서 구체적인 계산에서 sin 함수를 사용\n\n[sin 함수의 고차 미분]\n▪ 2차 미분뿐 아니라 3차 미분, 4차 미분도 계산\n▪ for 문을 사용하여 역전파를 반복하여, n차 미분을 구함\n▪ 먼저 gx = x.grad에서 미분값을 꺼내 gx에서 역전파하는 것임\n▪ 역전파를 하기 전에 x.cleargrad( )를 호출하여 미분값을 재설정함\n▪ 이 작업을 반복하여 n차 미분을 계산\n\n[sin 함수의 고차 미분 그래프 그리기]\n▪ x = Variable(np.linspace(-7, 7, 200)) 형태로 설정.\n• -7 부터 7까지 균일하게 200 등분한 배열을 만들어 줌.\n• 만들어진 1차원 배열을 Variable로 감싸 x에 할당.\n▪ 다차원 배열을 입력받으면 각 원소에 대해 독립적으로 계산함.\n▪ 한 번의 순전파로 원소 200개의 계산이 모두 이루어짐.\n▪ 각각의 그래프는 sin(x) -> cos(x)

In [54]:
# 고차 미분에 대응하는 새로운 Sin 클래스 구현
import numpy as np
from dezero.core import Function

class Sin(Function):
    def forward(self, x):
        y = np.sin(x)
        return y

    def backward(self, gy):
        x, = self.inputs
        gx = gy * cos(x)
        return gx

def sin(x):
    return Sin()(x)

In [55]:
# 고차 미분에 대응하는 새로운 Cos 클래스와 cos 함수 구현
import numpy as np
from dezero.core import Function

class Cos(Function):
    def forward(self, x):
        y = np.cos(x)
        return y

    def backward(self, gy):
        x, = self.inputs
        gx = gy * -sin(x)
        return gx

def cos(x):
    return Cos()(x)

In [56]:
# sin 함수의 고차 미분
import numpy as np
from dezero import Function
import dezero.functions as F

x = Variable(np.array(1.0))
y = F.sin(x)
y.backward(create_graph=True)

for i in range(3):
    gx = x.grad
    x.cleargrad()
    gx.backward(create_graph=True)
    print(x.grad)

TypeError: <class 'dezero.core_simple.Variable'> is not supported

In [57]:
# tanh 함수의 미분 구현
import numpy as np
from dezero import Function

class Tanh(Function):
    def forward(self, x):
        xp = cuda.get_array_module(x)
        y = xp.tanh(x)
        return y

    def backward(self, gy):
        y = self.outputs[0]()  # weakref
        gx = gy * (1 - y * y)
        return gx


def tanh(x):
    return Tanh()(x)

In [58]:
# tanh 함수의 고차 미분 계산 그래프 시각화
import numpy as np
from dezero import Variable
from dezero.utils import plot_dot_graph
import dezero.functions as F
from dezero.core_simple import Variable

x = Variable(np.array(1.0))
y = F.tanh(x)
x.name = 'x'
y.name = 'y'
y.backward(create_graph=True)

iters = 0

for i in range(iters):
    gx = x.grad
    x.cleargrad()
    gx.backward(create_graph=True)

gx = x.grad
gx.name = 'gx' + atr(iters + 1)
plot_dot_graph(x, verbose=False, to_file='tanh.png')

TypeError: <class 'dezero.core_simple.Variable'> is not supported

In [59]:
'''

<고차미분 이외의 용도>

[고차 미분 계산 정리]
▪ 고차 미분을 하기 위해 역전파 시 수행되는 계산에 대해서도 연결을 만들도록 함
▪ 역전파의 계산 그래프를 만들수 있음
▪ 고차 미분 외에 어떻게 활용할 수 있는지를 살펴봄

[Double Backpropagation]
▪ 역전파로 수행한 계산에 대해 또 다시 역전파를 수행함
▪ Double backprop 은 현대적인 딥러닝 프레임워크 대부분이 지원함

[double backprop 활용 용도]
▪ 미분이 포함된 식에서 다시 한번 미분 수행
▪ y’ = 2x, z = (𝑦′)^3 + y = 8 𝒙^3 + 𝒙^2
▪ z’ = 24 𝒙^2 + 2𝑥
▪ a, b 가 정수일 때 f(𝒙𝟎, 𝒙𝟏) = 𝒃(𝒙𝟏 - 𝒙𝟎^2)^2 + (a − 𝒙𝟎)^2

[DeZero를 사용하여 문제 계산]
▪ y.backward(create_graph=True) 는 미분을 하기 위한 역전파 코드이며 계산 그래프를 생성함.
▪ 역전파가 만들어낸 계산 그래프를 사용하여 새로운 계산을 하고 다시 역전파함
• 미분식을 구하고, 그 식을 사용하여 계산 후 또 다시 미분하는 문제를 해결함

<딥러닝 연구에서의 사용 예>

[WGAN-GP 논문]
▪ 수식을 최적화하는 함수가 포함됨.
▪ 최적화하는 식에 기울기(텐서의 각 원소에 대한 미분 결과)가 들어 있음
▪ 이 기울기는 첫 번째 역전파에서 구할 수 있음
▪ 이 기울기를 사용하여 함수 L을 계산하고, 함수 L을 최적화하기 위해 두 번째 역전파 수행

->DeZero의 역전파를 수정하여 double backprop를 가능하게 되었으며 
다음엔 DeZero를 신경망용으로 변경시킬 것

'''

'\n\n<고차미분 이외의 용도>\n\n[고차 미분 계산 정리]\n▪ 고차 미분을 하기 위해 역전파 시 수행되는 계산에 대해서도 연결을 만들도록 함\n▪ 역전파의 계산 그래프를 만들수 있음\n▪ 고차 미분 외에 어떻게 활용할 수 있는지를 살펴봄\n\n[Double Backpropagation]\n▪ 역전파로 수행한 계산에 대해 또 다시 역전파를 수행함\n▪ Double backprop 은 현대적인 딥러닝 프레임워크 대부분이 지원함\n\n[double backprop 활용 용도]\n▪ 미분이 포함된 식에서 다시 한번 미분 수행\n▪ y’ = 2x, z = (𝑦′)^3 + y = 8 𝒙^3 + 𝒙^2\n▪ z’ = 24 𝒙^2 + 2𝑥\n▪ a, b 가 정수일 때 f(𝒙𝟎, 𝒙𝟏) = 𝒃(𝒙𝟏 - 𝒙𝟎^2)^2 + (a − 𝒙𝟎)^2\n\n[DeZero를 사용하여 문제 계산]\n▪ y.backward(create_graph=True) 는 미분을 하기 위한 역전파 코드이며 계산 그래프를 생성함.\n▪ 역전파가 만들어낸 계산 그래프를 사용하여 새로운 계산을 하고 다시 역전파함\n• 미분식을 구하고, 그 식을 사용하여 계산 후 또 다시 미분하는 문제를 해결함\n\n<딥러닝 연구에서의 사용 예>\n\n[WGAN-GP 논문]\n▪ 수식을 최적화하는 함수가 포함됨.\n▪ 최적화하는 식에 기울기(텐서의 각 원소에 대한 미분 결과)가 들어 있음\n▪ 이 기울기는 첫 번째 역전파에서 구할 수 있음\n▪ 이 기울기를 사용하여 함수 L을 계산하고, 함수 L을 최적화하기 위해 두 번째 역전파 수행\n\n->DeZero의 역전파를 수정하여 double backprop를 가능하게 되었으며 \n다음엔 DeZero를 신경망용으로 변경시킬 것\n\n'